# Likelihood computation benchmarks
In this notebook we compare the run time of the various likelihood functions implemented in the recombulator-x module.

In [1]:
import time
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
from tqdm import tqdm
import itertools

In [2]:
import recombulatorx
from recombulatorx import compute_family_likelihood
from recombulatorx.testing import generate_processed_family, generate_random_rates

Retrieve the available implementations, which vary depending on the availability of the numba module.

In [3]:
available_implementations = recombulatorx.likelihood.implementations.keys()

Define benchmarking parameters, the number of families and the maximum mean time (in seconds) allowed for one family likelihood computation:

In [4]:
max_time = 1
n_fam_I = 100
n_fam_II = 100

In [5]:
time_df_acc = {}
slow_funcs = set()
first_run = True
for n_markers in tqdm(itertools.chain(range(2, 101), range(10**4 - 50, 10**4 + 1))):

    rates = generate_random_rates(n_markers)
    direct_numpy_cache = None        
                
    fams_I = [generate_processed_family(f'FAM_I_{i}', 2, True, *rates) for i in range(n_fam_I)]
    fams_II = [generate_processed_family(f'FAM_II_{i}', 2, False, *rates) for i in range(n_fam_II)]
    time_acc = {}
    if first_run: # run every function once, this is mainly for numba since it needs to compile!
        for impl in available_implementations:
            compute_family_likelihood(fams_I[0], *rates, implementation=impl)
            compute_family_likelihood(fams_II[0], *rates, implementation=impl)
        first_run = False
    
    for fams in fams_I, fams_II:
        for impl in available_implementations:
            for fam in fams:
                ftype = 'type I' if fam.is_mother_phased else 'type II'
                if (impl, ftype) in slow_funcs: continue
                t0 = time.time()
                lh = compute_family_likelihood(fam, *rates, implementation=impl)
                t1 = time.time()
                time_acc[(impl, ftype, fam.fid)] = [lh, t1 - t0]
    if len(time_acc) == 0:
        break
    
    df = pandas.DataFrame(time_acc, index=['Family Likelihood', 'Elapsed Time']).T
    df.index.names = ['Implementation', 'Family Type', 'Family ID']

    # check if any function is taking too long
    mean_dt = df.reset_index().groupby(['Implementation', 'Family Type'])['Elapsed Time'].mean()
    new_slow = set(mean_dt.index[mean_dt > max_time])
    slow_funcs |= new_slow
    if new_slow:
        print('dropped', new_slow, 'at', n_markers, 'markers')
    time_df_acc[n_markers] = df.reset_index()

0it [00:07, ?it/s]


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'inheritance_vectors': Cannot determine Numba type of <class 'function'>

File "recombulatorx/likelihood_direct.py", line 29:
def compute_phased_family_likelihood_direct_loop(mother, maternal_haplotypes, recombination_rates, mutation_rates):
    <source elided>
        son_lh = 0.0
        for v in inheritance_vectors(n):
        ^


In [ ]:
full_df = pandas.concat(time_df_acc, names=['# of markers', 'row']).reset_index('row', drop=True).reset_index()
full_df

In [ ]:
full_df.to_csv('benchmarks.tsv', index=False, sep='\t')

In [ ]:
full_df = pandas.read_csv('benchmarks.tsv', sep='\t')

## Plotting times

In [ ]:
fig = plt.figure(figsize=(15, 8))
seaborn.lineplot(data=full_df, x='# of markers', y='Elapsed Time', hue='Implementation', style='Family Type')
plt.yscale('log')

In [ ]:
fig = plt.figure(figsize=(10, 6))
seaborn.lineplot(data=full_df.loc[full_df['# of markers'] <= 36], x='# of markers', y='Elapsed Time', hue='Implementation', style='Family Type')
plt.yscale('log')

In [ ]:
fig.savefig('likelihood_benchmarks_1-35.pdf')

In [ ]:
plot_df = full_df.copy()
plot_df['BLOCK'] = float('nan')
plot_df.loc[plot_df['# of markers'] < 26, 'BLOCK'] = 'LEFT'
plot_df.loc[plot_df['# of markers'] > 100, 'BLOCK'] = 'RIGHT'

In [ ]:
fig = plt.figure(figsize=(10, 6))
seaborn.lineplot(
    data=plot_df.query('BLOCK == "LEFT"'), x='# of markers', y='Elapsed Time', 
    hue='Implementation', style='Family Type', 
)
plt.yscale('log')

In [ ]:
fig.savefig('likelihood_benchmarks_1-25.pdf')

In [ ]:
fig = seaborn.relplot(data=plot_df, x='# of markers', y='Elapsed Time', 
                hue='Implementation', style='Family Type', 
                col='BLOCK', kind='line', facet_kws={'sharey': True, 'sharex': False},
)
plt.yscale('log')

In [ ]:
fig.savefig('likelihood_benchmarks_split.pdf')

In [ ]:
assert False

In [ ]:
Qui facciamo dei test per vedere se la versione approssimata funziona?

In [ ]:
full_df.groupby(['n_markers', 'fam', 'rep'])['lh'].std()

In [ ]:
acc = {}
for l, sdf in full_df.groupby(['n_markers', 'fam', 'rep']):
    if len(sdf):
        sdf['rel_lh'] = sdf['lh']/sdf.query('func != "emp"')['lh'].max()
        acc[l] = sdf
full_df_r = pandas.concat(acc)

In [ ]:
full_df_r['rel_lh'].min()

In [ ]:

acc = {}
for l, sdf in full_df.groupby(['n_markers', 'fam', 'rep']):
    if len(sdf):
        sdf['rel_lh'] = sdf['lh']/sdf.query('func != "emp"')['lh'].max()
        acc[l] = sdf
full_df_r = pandas.concat(acc)

fig = plt.figure(figsize=(10, 5))
seaborn.lineplot(data=full_df_r, x='n_markers', y='rel_lh', hue='func', style='fam')
plt.ylim((0, 1.1))
plt.title('likelyhood accuracy')
pdf.savefig()
plt.close()